# NYC Taxi EDA using Koalas Library

Exploratory data analysis of the NYC Taxi dataset using Koalas. Looking at data quality and data distributions. This should be done over a larger sample but only looking at one month extract for performance. 

In [ ]:
from pyspark.sql import SparkSession
import databricks.koalas as ks
import os
import glob

# file = [x for x in glob.glob('/data/*.parquet')]
file = '/data/yellow_tripdata_2019-01.parquet'

spark = SparkSession.builder\
        .master("local")\
        .appName("NYC")\
        .getOrCreate()

In [ ]:
df_small = ks.read_parquet(file)
df_small.head()

In [ ]:
# Identify datatypes
df_small.dtypes

In [ ]:
# Convert pick/dropoff to datetimes

df_small['tpep_pickup_datetime'] = ks.to_datetime(df_small['tpep_pickup_datetime'])
df_small['tpep_dropoff_datetime'] = ks.to_datetime(df_small['tpep_dropoff_datetime'])

# Extract finer grain time details

df_small["pickup_month"] = df_small["tpep_pickup_datetime"].apply(lambda x: x.month)
df_small["pickup_day"] = df_small["tpep_pickup_datetime"].apply(lambda x: x.day)
df_small["pickup_weekday"] = df_small["tpep_pickup_datetime"].apply(lambda x: x.weekday())
df_small["pickup_hour"] = df_small["tpep_pickup_datetime"].apply(lambda x: x.hour)
df_small["pickup_minute"] = df_small["tpep_pickup_datetime"].apply(lambda x: x.minute)
df_small["pickup_time"] = df_small["pickup_hour"] + (df_small["pickup_minute"] / 60)

# df_small["dropoff_hour"] = df_small["tpep_dropoff_datetime"].apply(lambda x: x.hour)

# No trip duration column so calulcate that
df_small['trip_duration'] = (df_small['tpep_dropoff_datetime'] - df_small['tpep_pickup_datetime'])/60

In [ ]:
# Only keep columns that contain relevant information. Sourced from data dictionary
keep = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'trip_distance', 'PULocationID', 'DOLocationID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'pickup_month', 'pickup_day', 'pickup_weekday',
       'pickup_hour', 'pickup_minute', 'pickup_time', 'trip_duration']

df_small = df_small[keep]

In [ ]:
# Look at distribution of columns to identify any data quality issues
df_small.describe()

In [ ]:
# Drop rows with negative duration and anything longer than 20mins (75%)
df_small = df_small[df_small['trip_duration'].between(0,20)]
len(df_small)

In [ ]:
df_small['trip_duration'].plot.hist(bins=10)

In [ ]:
df_small['passenger_count'].plot.hist(bins=6)

In [ ]:
df_small['trip_distance'].plot.hist(bins=6)

In [ ]:
df_small['pickup_weekday'].plot.hist(bins=7)

In [ ]:
df_small['pickup_hour'].plot.hist(bins=24)